In [1]:
import math
import random
import pandas as pd
import numpy as np

from mesa import Agent
from mesa import Model
from mesa.datacollection import DataCollector
from mesa.space import Grid
from mesa.time import RandomActivation

In [2]:
def create_tx(from_address, nonce, start, end, gas_price, num_bytes):
    
    tx = {}
        
    tx['nonce'] = nonce
    tx['from_address'] = from_address # << Sensor ID
    tx['start_sync'] = start
    tx['end_sync'] = end
    tx['gas_price'] = gas_price
    # self.gas_limit = gas_limit # << unused parameter
    tx['num_bytes'] = num_bytes

    return tx

In [235]:
class Sensor(Agent):
    
    def __init__(self, unique_id, battery_life, 
                record_cost, record_freq, record_bytes, 
                compute_cost_per_byte, info_reduction,
                sign_cost,
                transmit_cost_per_byte, transmit_freq,
                blockchain, model):
        
        super().__init__(unique_id, model)
        
        if self.model.verbose:
            print('Creating Sensor agent ID', unique_id)
           
        self.unique_id = unique_id
        self.battery_life = battery_life
        self.dead = False
        self.record_cost = record_cost
        self.record_freq = record_freq
        self.record_bytes = record_bytes
        self.compute_cost_per_byte = compute_cost_per_byte
        self.info_reduction = info_reduction
        self.sign_cost = sign_cost
        self.transmit_cost_per_byte = transmit_cost_per_byte
        self.transmit_freq = transmit_freq
        self.blockchain = blockchain
        self.gwei_spent = 0
        self.data_collected = 0
        self.last_sync = 0
        self.nonce = 0
        self.db = np.array([])
        
        self.blockchain.chain.loc[0, self.unique_id] = False

    
    def record(self):

        if self.model.schedule.steps % self.record_freq == 0:
            
            self.battery_life -= self.record_cost
            self.db = np.append(self.db, self.record_bytes)
            
        else:
            self.db = np.append(self.db, 0)

    
    def compute(self, num_bytes):
        
        # Only invoked from within the transmit() method
        
        if self.info_reduction is not 1:
            self.battery_life -= self.compute_cost_per_byte * num_bytes
            return math.ceil(self.info_reduction * num_bytes)
        else:
            return num_bytes
            # with no compute cost
    
    def sign(self):
        
        # Only invoked from within the transmit() method
        self.battery_life -= self.sign_cost
        self.nonce += 1
    
    def next_nonce(self):
        self.nonce += 1
        return self.nonce
    
    def transmit(self):
        # Prepare data for transmission:
        
        # Calculate number of bytes to transmit (edge compute)
        bytes_collected = np.sum(self.db[self.last_sync : ])
        num_bytes_to_transmit = self.compute(bytes_collected)
        
        tx = create_tx(self.unique_id, self.next_nonce(), self.last_sync, self.model.schedule.steps, 20, num_bytes_to_transmit)
        
        # Prepare and sign tx
        self.sign()
        
        # Transmit, subtracting energy cost and adding gwei cost
        self.battery_life -= self.transmit_cost_per_byte * num_bytes_to_transmit
        self.blockchain.add_to_mempool(tx)
        
        self.last_sync = self.model.schedule.steps
    
    def confirm_tx(self, tx):
        
        self.gwei_spent += tx.gas_spend
        

    def step(self):
        
        if not self.dead:
            
            if self.battery_life < 0 and self.battery_life is not -999:
                if self.model.verbose:
                    print("Sensor", self.unique_id, "out of battery at tick", self.model.schedule.steps)
                self.dead = True
                pass

            self.record()    

            if self.transmit_freq >= 1:
                if self.model.schedule.steps % self.transmit_freq == 0:
                    print("TRansmitting", self.model.schedule.steps,'%', self.transmit_freq, "==", self.model.schedule.steps % self.transmit_freq)
                    self.transmit()
            elif self.transmit_freq > random.random():
                self.transmit()



In [234]:
2 % 7

2

In [188]:
class Blockchain(Agent):
    
    def __init__(self, unique_id, gas_price,
                gas_per_byte, gas_per_second, model):
        
        super().__init__(unique_id, model)
        
        if self.model.verbose:
            print("Blockchain created: ID", unique_id)
        
        self.unique_id = unique_id
        self.gas_price = gas_price
        self.gas_per_byte = gas_per_byte
        self.gas_per_second = gas_per_second
        self.chain = pd.DataFrame()
        
        self.tx_ct = 0
        self.mempool = pd.DataFrame(columns=["from_address", "nonce", 
                                             "start_sync", "end_sync", 
                                             "gas_price", "num_bytes", 
                                             "gas_spend", "tx_id",
                                             "mined", "block_submitted"])

    def add_to_mempool(self, tx):
        tx['gas_spend'] = tx['gas_price'] * self.gas_per_byte * tx['num_bytes']

        tx['tx_id'] = self.tx_ct
        tx['mined'] = False
        tx['block_submitted'] = self.model.schedule.steps
        row = pd.DataFrame(tx, index = [self.tx_ct])
        
        self.tx_ct += 1
        self.mempool = self.mempool.append(row, ignore_index=True)
    
    def write_data(self, num_bytes):

        gwei_spent = self.gas_per_byte * num_bytes * self.gas_price
        return gwei_spent

    def compute(self, num_seconds):
        gwei_spent = self.gas_per_second * num_seconds * self.gas_price
        return gwei_spent
    
    def mine_block(self):
        
        print("BLOCK NUMBER:", self.model.schedule.steps)
        self.chain.loc[self.model.schedule.steps] = [False for col in self.chain.columns]

        # Sort mempool to get highest-value transactions
        mp = self.mempool[self.mempool['mined'] == False].sort_values(by=['gas_spend']).reset_index()
        
        if len(mp) > 0:
            mp['cum_gas'] = mp['gas_spend'].cumsum()
            
#             print("With cumulative gas spend", mp)
#             print("Up to index", mp[mp['cum_gas'] > 2000000].index[0])

            tx_mined = mp[0 : mp[mp['cum_gas'] > 2000000].index[0] - 1] # << Tx to mine
#             print('Tx to mine:', tx_mined)
            
            print("Mining", len(tx_mined), "out of", len(mp), "unvalidated transactions.")
            print("Gas value:", tx_mined['gas_spend'].sum())
            
            for tx in tx_mined.iterrows():
                
                print("Mining tx id:", tx[1].tx_id)
                self.model.schedule._agents[tx[1].from_address].confirm_tx(tx[1])
                self.mempool.loc[self.mempool['tx_id'] == tx[1].tx_id, "mined"] = True
                self.mempool.loc[self.mempool['tx_id'] == tx[1].tx_id, "block_mined"] = self.model.schedule.steps
                
                self.chain.loc[tx[1].start_sync : tx[1].end_sync, tx[1].from_address] = True
                
        else:
            print('Empty mempool')
            pass
        
        
        
#         self.next_block = 
        # Select tx up to 4712388 gas limit (https://medium.com/@piyopiyo/how-to-get-ethereum-block-gas-limit-eba2c8f32ce)
        
        # Loop through transactions 
            # Deduct gwei from sensor wallet
            # 
        
        # Set mempool 

In [221]:
class SensorBlockchainNetwork(Model):
    
    def __init__(self, num_sensors,
                verbose=True):
        
        super().__init__()
        
        self.verbose = verbose
        if self.verbose:
            print('Verbose model')
        
        self.running = True
        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(
                                model_reporters = {
                                    "active_sensors": lambda m: len(m.schedule._agents),
#                                     "blockchain_size": lambda m: m.blockchain.chain_size # in batch run? 
                                },
                                agent_reporters = {
                                    "gwei_spent": lambda a: a.gwei_spent,
                                    "battery_life": lambda a: a.battery_life,
                                    "data_collected": lambda a: a.data_collected
                                })
        
        self.blockchain = Blockchain(self.next_id(), 10,
                                    625, 75000000, self) 
                                    # calculated based on 
                                    # https://hackernoon.com/ether-purchase-power-df40a38c5a2f
        
        for i in range(num_sensors):
            sensor = Sensor(self.next_id(), 1000, 
                            1, 1, 32,
                            1, 1, 0.1,
                            1, 7,
                            self.blockchain,
                            self)
            
            self.schedule.add(sensor)
    
    
        # Mine genesis block
        self.blockchain.chain.loc[1] = [False for col in self.blockchain.chain.columns]

        if self.verbose:
            print(num_sensors, "instantiated and added to schedule.")
    
    def step(self):
        self.schedule.step()
        if self.verbose:
            print("Mining block:", self.schedule.steps)
        if self.schedule.steps > 1:
            self.blockchain.mine_block()
        self.datacollector.collect(self)

In [236]:
model = SensorBlockchainNetwork(10, True)

Verbose model
Blockchain created: ID 1
Creating Sensor agent ID 2
Creating Sensor agent ID 3
Creating Sensor agent ID 4
Creating Sensor agent ID 5
Creating Sensor agent ID 6
Creating Sensor agent ID 7
Creating Sensor agent ID 8
Creating Sensor agent ID 9
Creating Sensor agent ID 10
Creating Sensor agent ID 11
10 instantiated and added to schedule.


In [240]:
model.step()

Mining block: 4
BLOCK NUMBER: 4


IndexError: index 0 is out of bounds for axis 0 with size 0

In [241]:
model.blockchain.chain

2      3      4      5      6      7      8      9      10     11
0   True   True  False   True   True   True   True   True   True  False
1  False  False  False  False  False  False  False  False  False  False
2  False  False  False  False  False  False  False  False  False  False
3  False  False  False  False  False  False  False  False  False  False
4  False  False  False  False  False  False  False  False  False  False

In [242]:
model.blockchain.mempool

block_submitted end_sync from_address gas_price  gas_spend  mined nonce  \
0               0        0            5        20   400000.0   True     1   
1               0        0            2        20   400000.0   True     1   
2               0        0            3        20   400000.0   True     1   
3               0        0            6        20   400000.0   True     1   
4               0        0            9        20   400000.0   True     1   
5               0        0           10        20   400000.0   True     1   
6               0        0            7        20   400000.0   True     1   
7               0        0            8        20   400000.0   True     1   
8               0        0           11        20   400000.0  False     1   
9               0        0            4        20   400000.0  False     1   

   num_bytes start_sync tx_id  block_mined  
0       32.0          0     0          2.0  
1       32.0          0     1          2.0  
2       32.0          0     2          2.0  
3       32.0          0     3          2.0  
4       32.0          0     4          3.0  
5       32.0          0     5          3.0  
6       32.0          0     6          3.0  
7       32.0          0     7          3.0  
8       32.0          0     8          NaN  
9       32.0          0     9          NaN

In [243]:
# for i in range(10):
model.step()


Mining block: 5
BLOCK NUMBER: 5


IndexError: index 0 is out of bounds for axis 0 with size 0

In [186]:
mp = model.blockchain.mempool

In [187]:
mp

block_mined block_submitted end_sync from_address gas_price  gas_spend  \
0           2.0               0        0            8        20   400000.0   
1           3.0               0        0            5        20   400000.0   
2           3.0               0        0           10        20   400000.0   
3           3.0               0        0            2        20   400000.0   
4           3.0               0        0           11        20   400000.0   
5           4.0               0        0            6        20   400000.0   
6           2.0               0        0            3        20   400000.0   
7           2.0               0        0            9        20   400000.0   
8           2.0               0        0            4        20   400000.0   
9           4.0               0        0            7        20   400000.0   
10          5.0               1        1            4        20   800000.0   
11          6.0               1        1            2        20   800000.0   
12          7.0               1        1            5        20   800000.0   
13          NaN               1        1            7        20   800000.0   
14          NaN               1        1            6        20   800000.0   
15          NaN               1        1           10        20   800000.0   
16          NaN               1        1            9        20   800000.0   
17          NaN               1        1           11        20   800000.0   
18          NaN               1        1            8        20   800000.0   
19          NaN               1        1            3        20   800000.0   
20          NaN               2        2            6        20   800000.0   
21          NaN               2        2            8        20   800000.0   
22          NaN               2        2            3        20   800000.0   
23          NaN               2        2           10        20   800000.0   
24          NaN               2        2            9        20   800000.0   
25          NaN               2        2            4        20   800000.0   
26          NaN               2        2            7        20   800000.0   
27          NaN               2        2           11        20   800000.0   
28          NaN               2        2            5        20   800000.0   
29          NaN               2        2            2        20   800000.0   
..          ...             ...      ...          ...       ...        ...   
40          NaN               4        4            2        20   800000.0   
41          NaN               4        4            9        20   800000.0   
42          NaN               4        4           10        20   800000.0   
43          NaN               4        4            4        20   800000.0   
44          NaN               4        4            7        20   800000.0   
45          NaN               4        4            8        20   800000.0   
46          NaN               4        4            6        20   800000.0   
47          NaN               4        4            5        20   800000.0   
48          NaN               4        4            3        20   800000.0   
49          NaN               4        4           11        20   800000.0   
50          NaN               5        5           11        20   800000.0   
51          NaN               5        5            8        20   800000.0   
52          NaN               5        5            2        20   800000.0   
53          NaN               5        5            9        20   800000.0   
54          NaN               5        5            5        20   800000.0   
55          NaN               5        5            7        20   800000.0   
56          NaN               5        5           10        20   800000.0   
57          NaN               5        5            3        20   800000.0   
58          NaN               5        5            4        20   800000.0   
59          NaN           

In [ ]:
mp['cum_gas'] = mp['gas_spend'].cumsum()

In [ ]:
mp[0:mp[mp['cum_gas'] > 4712388].index[0]]

In [ ]:
a_df = model.datacollector.get_agent_vars_dataframe()
m_df = model.datacollector.get_model_vars_dataframe()

In [ ]:
m_df

In [ ]:
a_df

In [ ]:
# Batch Run

batch_runner = BatchRunner(
    SensorBlockchainNetwork,
    variable_params,
    fixed_params,
    iterations = 5,
    max_steps = 100,
    model_reporters = {"blockchain_size": lambda m: m.blockchain.chain_size}
)